## General principles

Conceptually, aggregation/disaggregation operations are

* start with *indexed data* (index can be multidimensional)
* use a *weight map* to map data to a new (multidimensional) index.
* group values for each unique in the new index and use a *weighted aggregation*, 
* which depends on the *variable type*, e.g.nominal, ordinal, numerical (intensive, extensive)


In [1]:
## Examples Using pandas

# although not required, we use pandas Series for data and weights using named Index/MultiIndex 
# We start by setting up some dimensions (2 spatial, 1 temporal) using named Index

from pandas import Series, Index, MultiIndex
from data_disaggregation import VT_Numeric, VT_NumericExt

dim_region = Index(["r1", "r2"], name="region")
dim_subregion = Index(["r11", "r12", "r21", "r22"], name="subregion")
dim_time = Index(["t1", "t2", "t3"], name="time")

# We can use MultiIndex to create cross products:

dim_region_subregion = MultiIndex.from_product([dim_region, dim_subregion])
dim_region_time = MultiIndex.from_product([dim_region, dim_time])
dim_region_time

MultiIndex([('r1', 't1'),
            ('r1', 't2'),
            ('r1', 't3'),
            ('r2', 't1'),
            ('r2', 't2'),
            ('r2', 't3')],
           names=['region', 'time'])

In [2]:
# now we create Series for data and weights (which also includes relationships between dimensions)
# using a value of 1 here because all the subregions have the same weight relatively
w_region_subregion = Series({("r1", "r11"): 1, ("r1", "r12"): 1, ("r2", "r21"): 1, ("r2", "r22"): 1}, index=dim_region_subregion)

# define some data on the regional level
d_region = Series({"r1": 100, "r2": 200}, index=dim_region)

# use extensive disaggregation:
d_subregion = VT_NumericExt.disagg(d_region, w_region_subregion)
d_subregion

subregion
r11     50
r12     50
r21    100
r22    100
dtype: int64

In [3]:
# applying the same weight map aggregates it back.
VT_NumericExt.disagg(d_subregion, w_region_subregion)

region
r1    100
r2    200
dtype: int64

In [4]:
# using Intensive distribution, the values for the regions in the disaggregation are duplicated
VT_Numeric.disagg(d_subregion, w_region_subregion)

region
r1     50
r2    100
dtype: int64

In [5]:
# distribute over a new dimension (time)
w_time = Series({"t1": 2, "t2": 3, "t3": 5}, index=dim_time)
VT_NumericExt.disagg(d_region, w_time, dim_region_time)

region  time
r1      t1       20
        t2       30
        t3       50
r2      t1       40
        t2       60
        t3      100
dtype: int64

In [6]:
# what about scalar
s_time = VT_NumericExt.disagg(100, w_time)
s_time

time
t1    20
t2    30
t3    50
dtype: int64

In [7]:
VT_NumericExt.disagg(s_time, w_time)

100